# Imports, setting up device and seeds

In [1]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
 
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# torch.cuda.set_device(1)
# device = torch.device("cuda")
device = torch.device('mps')
# device = torch.device('cpu')

In [3]:
# torch.manual_seed(42)


import os
import random

def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_all_seeds(42)

# Constants

In [4]:
GOOGLE_COLAB_VERSION = False
VALIDATION_PERCENTAGE = 0.10
TRAIN_PATH = "./train"
TEST_PATH = "./test_all"

# Get data

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainFolder = torchvision.datasets.ImageFolder(root=TRAIN_PATH,
                                               transform=transform)

# trainloader = torch.utils.data.DataLoader(trainFolder, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root=TEST_PATH, train=False,
#                                        download=False, transform=transform)

# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)

In [6]:
# split into train and validation
train_size = int((1 - VALIDATION_PERCENTAGE) * len(trainFolder))
val_size = len(trainFolder) - train_size
train_subset, val_subset = torch.utils.data.random_split(trainFolder, [train_size, val_size])

In [7]:
len(train_subset.indices)

79209

In [8]:
len(val_subset.indices)

8802

In [27]:
# create dataloaders
train_dataloader = torch.utils.data.DataLoader(train_subset, batch_size=256, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_subset, batch_size=256) # TODO: change to max, który się szybko liczy

In [10]:
classes = os.listdir(TRAIN_PATH)

In [11]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        ## Warstwa konwolucyjna
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=0)
        ## Warstwa max pooling 
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 256, 3)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(1000, 512)
        self.fc2 = nn.Linear(512, 50)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [12]:
import torchvision.models as models

# net = Net().to(device)
net = models.resnet18().to(device)
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [28]:
def get_accuracy(val_dataloader=val_dataloader):
    # copy and create new get_accuracy() func - per class
    val_correct_count = 0
    val_data_len = 0

    for i, data in enumerate(val_dataloader, 0):
        net.eval()
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # forward + backward + optimize
        outputs = net(inputs)

        outputs_for_val_correct_count = outputs.max(1, keepdim=True)[1].squeeze()
        val_correct_count += (outputs_for_val_correct_count == labels).sum()
        val_data_len += len(labels)

    new_val_acc = val_correct_count / val_data_len
    return new_val_acc

In [15]:
%%time

epochs, losses, train_acc, val_acc = [], [], [], []

NUM_EPOCHS = 1
for epoch in range(NUM_EPOCHS): 
    # epoch_loss = 0
    # train_accuracy = 0
    # val_accuracy = 0
    train_correct_count = 0
    train_data_len = 0
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        net.train()
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # zero the parameter gradients
        optimizer.zero_grad()

        # print statistics
        # epoch_loss += float(loss)
        # train_accuracy += get_accuracy(net, train_subset)

        # val_accuracy_single_iter = get_accuracy(net, val_subset)
        # val_accuracy += val_accuracy_single_iter
        # print(len(outputs.max(1, keepdim=True)[1]))
        # print(len(labels))
        outputs_for_train_correct_count = outputs.max(1, keepdim=True)[1].squeeze()
        # print(outputs_for_train_correct_count)
        train_correct_count += (outputs_for_train_correct_count == labels).sum()
        # print(train_correct_count)
        # print(len(labels))
        train_data_len += len(labels)

        running_loss_single_iter = loss.item()
        running_loss += running_loss_single_iter
        # print(i, running_loss_single_iter)

    # losses.append(epoch_loss/(batch_size * len(train_loader)))
    # epochs.append(n)
    # new_train_acc = 0 # train_accuracy/len(train_loader)
    # new_train_acc = get_accuracy(net, train_subset)
    # train_acc.append(new_train_acc)
    # new_val_acc = val_accuracy/len(train_dataloader)
    # val_acc.append(new_val_acc)
    new_train_acc = train_correct_count / train_data_len
    train_acc.append(new_train_acc)
    
    # if n%5==0:
    # print(f"Epoch {epoch}, train_acc={round(new_train_acc, 2)}, val_acc={round(new_val_acc, 2)}")
    print(f'epoch {epoch} train accuracy = {new_train_acc}, val_accuracy = {get_accuracy()}')

    print(f'[{epoch+1}/{NUM_EPOCHS}] loss: {running_loss/1000}')
    running_loss = 0.0

print('Finished Training')

epoch 0 train accuracy = 0.16594073176383972, val_accuracy = 0.2605089843273163
[1/1] loss: inf
Finished Training
CPU times: user 27.9 s, sys: 7.68 s, total: 35.5 s
Wall time: 1min 18s


In [31]:
print(f'validation accuracy = {get_accuracy()}')

validation accuracy = 0.2605089843273163
